In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install tweepy

Link to Twitter Dev Portal: https://developer.twitter.com/en/portal/projects/1342246467465965568/apps/19648638/keys

Thank you: https://gist.github.com/vickyqian/f70e9ab3910c7c290d9d715491cde44c

In [ ]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = 
consumer_secret = 
access_token = 
access_token_secret = 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####Healthcare Workers Burnout 
# Open/Create a file to append data
csvFile = open('burnout.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

created_at = []
text = []
for tweet in tweepy.Cursor(api.search,q=["#covidnurse"],count=100,
                           lang="en",
                           since="2020-03-15").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    created_at.append(tweet.created_at)
    text.append(tweet.text)

In [ ]:
df =  pd.DataFrame(
    {'created_at': created_at,
     'tweet': text
    })
df #RT (Retweets) are duplicates since Twitter is an echo chamber; will need to get rid of duplicates

In [ ]:
df_clean = df[~df.tweet.str.contains("RT")]
df_clean

In [ ]:
!pip3 install nlp-profiler

In [ ]:
from nlp_profiler.core import apply_text_profiling

new_text_column_dataset = apply_text_profiling(df_clean, 'tweet')
new_text_column_dataset

nlp_profiler is not accurate...will do own EDA and analysis on tweets

# Scrape multiple users and multiple hashtags/keywords from Twitter

 Thank you: https://github.com/MIA-GH/Elections/blob/master/scripts/main.py

In [ ]:
"""
    This script file extracts tweets from multiple nurses who have commented about their Pandemic experience. 
    Also, several key words/hashtags are used that have had a lot of information about toll the Pandemic has had on healthcare workers. 
"""

import tweepy
from tweepy import Cursor
import unicodecsv
from unidecode import unidecode
#from scripts import twitter_credentials as api

# Authentication and connection to Twitter API.
consumer_key =
consumer_secret = 
access_key = 
access_secret = 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


def main():
    # the Script targets any of these keywords in the loop
    terms = [
        "covidnurse", "RN", "", "nursescovid19", "nursetwitter", "moralinjury", "thosewecarry"
    ]

    # insert the keyword here for the extraction to continue
    retweet_filter = '-filter:retweets'

    # append the term to search parameters
    q = retweet_filter
    tweets_per_qry = 100
    since_id = None

    max_id = -1
    max_tweets = 1000

    tweet_count = 0

    # Usernames whose tweets we want to gather.
    users = [
        "rn_critcare",
     
    ]

    # giving the user some feed back that the script is running
    print("Downloading tweets from user timelines...")

    # extract tweets from timeline of targeted nurses/hashtags
    try:
        with open('burnout2.csv', 'ab') as file:
            writer = unicodecsv.writer(file, delimiter=',', quotechar='"')
            # Write header row.

            # write the titles for each column
            writer.writerow([
                "Tweet Date",
                "Tweet ID",
                "Tweet Text",
                "tweet_source",
                "tweet_retweet_count",
                "tweet_favorite_count"
            ])

            # loop through all the users and extract tweets from their relative timelines
            for user in users:

                # Get 1000 most recent tweets for the current user.
                for tweet in Cursor(api.user_timeline, screen_name=user).items(1000):
                    # Get info specific to the current tweet of the current user.
                    tweet_text = unidecode(tweet.text)

                    # format the date
                    tweet_date = str(tweet.created_at.year) + "/" + str(tweet.created_at.month) + "/" + str(
                        tweet.created_at.day)

                    tweet_source = tweet.source

                    retweet_count = tweet.retweet_count

                    favorite_count = tweet.favorite_count

                    tweet_id = tweet.id

                    # Write the extracted tweets to CSV.
                    writer.writerow([
                        tweet_date, tweet_id, tweet_text, tweet_source,
                        retweet_count, favorite_count,
                    ])

                # Show progress.
                print("Wrote tweets of %s to CSV." % user)

            print("streaming at least {0} tweets".format(max_tweets))
            for term in terms:
                search_query = term + q
                while tweet_count < max_tweets:
                    try:
                        if max_id <= 0:
                            if not since_id:
                                new_tweets = api.search(q=search_query, lang="en", count=tweets_per_qry,
                                                        tweet_mode='extended')

                            else:
                                new_tweets = api.search(q=search_query, lang="en", count=tweets_per_qry,
                                                        since_id=since_id, tweet_mode='extended')
                        else:
                            if not since_id:
                                new_tweets = api.search(q=search_query, lang="en", count=tweets_per_qry,
                                                        max_id=str(max_id - 1), tweet_mode='extended')
                            else:
                                new_tweets = api.search(q=search_query, lang="en", count=tweets_per_qry,
                                                        max_id=str(max_id - 1),
                                                        since_id=since_id, tweet_mode='extended')

                        if not new_tweets:
                            print("No more tweets found")
                            break
                        for tweet in new_tweets:
                            tweet_text = unidecode(tweet.full_text)

                            tweet_date = str(tweet.created_at.year) + "/" + str(tweet.created_at.month) + "/" + str(
                                tweet.created_at.day)

                            tweet_source = tweet.source

                            retweet_count = tweet.retweet_count

                            favorite_count = tweet.favorite_count

                            tweet_id = tweet.id
                            writer.writerow([
                                tweet_date, tweet_id, tweet_text, tweet_source,
                                retweet_count, favorite_count,
                            ])

                        tweet_count += len(new_tweets)
                        print("Downloaded {0} tweets".format(tweet_count))
                        max_id = new_tweets[-1].id

                    except tweepy.TweepError as e:
                        # Just exit if any error
                        print("some error : " + str(e))
                        break
            print("Downloaded at least {0} tweets, Saved to csv file".format(tweet_count))
    except tweepy.TweepError as e:
        print("There was an error, find details below, else check your internet connection or your " +
              " credentials in the credentials.py file \n")
        print("If this is not your first time running this particular script, then there is a possibility that the "
              "maximum rate limit has been exceeded. wait a few more minutes and re run the script.\n")
        print(f"Error Details: {str(e)}")


if __name__ == "__main__":
    main()

In [ ]:
df2 = pd.read_csv('burnout2.csv')
df2

In [ ]:
df2['Tweet Text'].str.contains("RT")

Retweets are present despite scrape programmed to filter them out. So will do a Pandas filter just to make sure to eliminate redundancy.

In [ ]:
df_clean2 = df2[~df2['Tweet Text'].str.contains("RT")]
df_clean2

In [ ]:
df_clean2.to_csv('df.csv')

Great! 1885 rows to use for training and testing an NLP model :)